In [1]:
import time
main_start = time.perf_counter()

import multiprocessing
import pandas as pd
import numpy as np
from datetime import datetime, date

# from sklearn.model_selection import train_test_split

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import GridSearchCV
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression


In [2]:
# Read the data
train = pd.read_csv('./sales_train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')
items = pd.read_csv('./items.csv')
item_cats = pd.read_csv('./item_categories.csv')
shops = pd.read_csv('./shops.csv')

In [3]:
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1)#.reset_index()

In [4]:

# start = time.perf_counter()

# train['day']= np.array(train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%d')), dtype=np.int8)
# train['month']= train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
# train['year']= train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
# finish = time.perf_counter()

# print(f'Finished in {round(finish-start, 2)} second(s)')

In [5]:
start = time.perf_counter()
processes = []
q = multiprocessing.Queue()
types = ['d','m','y']

def editDate(q,arg):
    obj = np.array(train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%'+arg)), dtype=np.int8)
    q.put([arg,obj])

for i in types:
    process=multiprocessing.Process(target=editDate, args=[q,i])
    process.start() 
    processes.append(process)

for _ in types:
    result = q.get()
    train[result[0]]=result[1]

for process in processes:
    process.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 48.16 second(s)


In [6]:
train

,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id,d,m,y
item_id,,,,,,,,,
0,01.09.2014,20,54,58.0,1.0,40,1,9,14
1,04.04.2014,15,55,4490.0,1.0,76,4,4,14
1,02.04.2014,15,55,4490.0,1.0,76,2,4,14
1,06.07.2014,18,55,4490.0,1.0,76,6,7,14
1,04.08.2014,19,55,4490.0,1.0,76,4,8,14
...,...,...,...,...,...,...,...,...,...
22168,02.02.2013,1,42,799.0,1.0,62,2,2,13
22168,17.02.2013,1,16,799.0,1.0,62,17,2,13
22168,08.03.2013,2,43,798.5,1.0,62,8,3,13


In [7]:
y = train['item_cnt_day']
train.drop(columns=['item_cnt_day', 'date'], axis=1, inplace=True)
# Obtain target and predictors
features = ['d','m', 'y', 'date_block_num','shop_id', 'item_price', 'item_category_id']

In [8]:
main_finish = time.perf_counter()

print(f'Finished in {round(main_finish-main_start, 2)} second(s)')

Finished in 51.2 second(s)
